In this chapter, we will cover the following topics:

<ul>
    <li>Punching holes in polygons with a symmetric difference operation</li>
    <li>Union polygons without merging</li>
    <li>Union polygons with merging (dissolving)</li>
    <li>Performing an identity function (difference + intersection)</li>

## Introduction

Discovering how two datasets spatially relate to each other when they are placed over one another is called overlay analysis. An overlay can be compared to a sheet of tracing paper. For example, you could overlay the tracing paper on top of your base map and see what areas overlap each other. This process is and was a game changer in spatial analysis and modeling. Computer-aided GIS computations can therefor automatically identify where two geometry sets spatially touch for example.

The goal of this chapter is to give you a feel for the most common overlay analysis functions, such as unions, intersects, and symmetrical differences. These are based on the Dimensionally Extended nine intersection model (DE-9IM), which can be found at http://en.wikipedia.org/wiki/DE-9IM, and describes our list of possible overlays. All processes that we use or name here are derived using a combination of these nine predicates.

<img src="./50790OS_06_01.jpg" height=400 width=400>

We will explore these topology rules in depth in Chapter 9, Topology Checking and Data Validation.

## 6.1. Punching holes in polygons with a symmetric difference operation

Why, oh why would we want to punch holes in polygons and create a donut? Well, this is done for several reasons, for example, you may want to remove a lake polygon from a forest polygon that it overlaps since it sits in the middle of the forest and is, therefore, included in your area calculations.

Another example is where we have a set of polygons representing a golf course's fairways and a second set of polygons representing the greens that overlap these fairways. Our task is to calculate the correct number of square meters of fairways. The greens will create our donuts in a fairway's polygons.

This is translated into spatial operation terminology and means that we need to perform a symmetric difference operation or, in ESRI terminology, an "erase" operation.

<img src="./50790OS_06_02.jpg" height=400 width=400>

### Getting ready

In this example, we will create two sets of visualizations to see our results. Our output will generate Well Known Text (WKT) that is displayed in your browser using the Openlayers 3 web mapping client.

For this example, make sure you have all your code downloaded in your /ch06 folder provided at GitHub and have this folder structure containing these files:

<pre><code>
code
¦   ch06-01_sym_diff.py
¦   foldertree.txt
¦   utils.py
¦
+---ol3
    +---build
    ¦       ol-debug.js
    ¦       ol-deps.js
    ¦       ol.js
    ¦
    +---css
    ¦       layout.css
    ¦       ol.css
    ¦
    +---data
    ¦       my_polys.js
    ¦
    +---html
    ¦       ch06-01_sym_diff.html
    ¦
    +---js
    ¦       map_sym_diff.js
    ¦
    +---resources
        ¦   jquery.min.js
        ¦   logo-32x32-optimized.png
        ¦   logo-32x32.png
        ¦   logo.png
        ¦   textured_paper.jpeg
        ¦
        +---bootstrap
            +---css
            ¦       bootstrap-responsive.css
            ¦       bootstrap-responsive.min.css
            ¦       bootstrap.css
            ¦       bootstrap.min.css
            ¦
            +---img
            ¦       glyphicons-halflings-white.png
            ¦       glyphicons-halflings.png
            ¦
            +---js
                    bootstrap.js
                    bootstrap.min.js

geodata
    pebble-beach-fairways-3857.geojson
    pebble-beach-greens-3857.geojson
    results_sym_diff.js
</code></pre>
    
With the folder structure in place, when you run the code, all inputs and outputs will find their correct home.

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import json
from os.path import realpath
from shapely.geometry import MultiPolygon
from shapely.geometry import asShape
from shapely.wkt import dumps


# define our files input and output locations
input_fairways = realpath("../geodata/pebble-beach-fairways-3857.geojson")
input_greens = realpath("../geodata/pebble-beach-greens-3857.geojson")
output_wkt_sym_diff = realpath("ol3/data/results_sym_diff.js")


# open and load our geojson files as python dictionary
with open(input_fairways) as fairways:
    fairways_data = json.load(fairways)

with open(input_greens) as greens:
    greens_data = json.load(greens)

# create storage list for our new shapely objects
fairways_multiply = []
green_multply = []

# create shapely geometry objects for fairways
for feature in fairways_data['features']:
    shape = asShape(feature['geometry'])
    fairways_multiply.append(shape)

# create shapely geometry objects for greens
for green in greens_data['features']:
    green_shape = asShape(green['geometry'])
    green_multply.append(green_shape)

# create shapely MultiPolygon objects for input analysis
fairway_plys = MultiPolygon(fairways_multiply)
greens_plys = MultiPolygon(green_multply)

# run the symmetric difference function creating a new Multipolygon
result = fairway_plys.symmetric_difference(greens_plys)

# write the results out to well known text (wkt) with shapely dump
def write_wkt(filepath, features):
    with open(filepath, "w") as f:
        # create a js variable called ply_data used in html
        # Shapely dumps geometry out to WKT
        f.write("var ply_data = '" + dumps(features) + "'")

# write to our output js file the new polygon as wkt
write_wkt(output_wkt_sym_diff, result)

Your output will be available in the /ch06/code/ol3/html/ folder with the ch06-01_sym_diff.html filename. Simply open this file in your local web browser, such as Chrome, Firefox, or Safari. Our output web map was created by modifying the Openlayers 3 example code pages according to our needs. The resulting web map should display the following map in your local web browser:

<img src="./50790OS_06_03.jpg" height=400 width=400>

You can now clearly see a hole inside our fairway.

### How it works...

To begin with, we use two GeoJSON datasets as our input, both with EPSG: 3857 and stemming from the OSM EPSG: 4326. The transformation process is not covered here; take a look at Chapter 2, Working with Projections, for further information on how to transform data between two coordinate systems.

Our first task is to read in both the GeoJSON files into Python dictionaries objects using the standard Python json module. Next, we set up some empty lists that will store the Shapely geometry objects as a list used for our input to generate the needed MultiPolygons for our analysis. We use the Shapely built-in asShape() function to create the Shapely geometry objects so that we can perform the spatial operations. This is accomplished by accessing the dictionaries' ['geometry'] element. We then append each geometry to our empty list. This list is then inputted into the Shapely MultiPolygon() function that will create a MultiPolygon for us and is used as our inputs.

The actual process of running our symmetric_difference happens when we input the fairways_plys MultiPolygon as input and the parameter passed is the greens_ply MultiPolygon. The output is stored in the result variable, which itself is also a MultiPolygon. Not to forget, a MultiPolygon is just a list of polygons that we can iterate over.

Next up, we'll take a look at a function called write_wkt(filepath, features). This outputs our resulting MultiPolygon Shapely geometry to the Well Known Text (WKT) format. We do not simply output this WKT but instead, create a new JavaScript file, ol3/data/ch06-01_results_sym_diff.js, containing our WKT output. The code outputs a string that creates a JavaScript variable called ply_data. This ply_data variable is then used in our HTML file located at /ch06/code/ol3/html/sym_diff.html to draw our WKT vector layer using Openlayers 3. We then call our function and it executes the write to the WKT JavaScript file.

This example is the first that visualizes our results as a web map. In Chapter 11, Web Analysis with GeoDjango, we will explore a fully functional web mapping application; for those of you who cannot wait, you may want to jump ahead. Further examples will continue to use Openlayers 3 as our data viewer, moving away from using Matplotlib.

In the end, our simple one-line symmetric difference execution needed a lot of helper code to deal with importing GeoJSON data and exporting the results in a format that could display a web map with Openlayers 3.

### 6.2. Union polygons without merging

To demonstrate what merging is all about, we will take an example from the National Oceanic and Atmospheric Administration (NOAA) weather data. It provides an awesome minute-by-minute update of Shapefiles for your desire to download data. We will look at a one-week collection of weather warnings, and combine these with state boundaries to see where exactly warnings occurred within a state boundary.

<img src="./50790OS_06_04.jpg" height=400 width=400>

The preceding screenshot shows us the polygons before the union operation in QGIS.


In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import json
from os.path import realpath
import shapefile  # pyshp
from geojson import Feature, FeatureCollection
from shapely.geometry import asShape, MultiPolygon
from shapely.ops import polygonize
from shapely.wkt import dumps


def create_shapes(shapefile_path):
    """
    Convert Shapefile Geometry to Shapely MultiPolygon
    :param shapefile_path: path to a shapefile on disk
    :return: shapely MultiPolygon
    """
    in_ply = shapefile.Reader(shapefile_path)

    # using pyshp reading geometry
    ply_shp = in_ply.shapes()
    ply_records = in_ply.records()
    ply_fields = in_ply.fields
    print ply_records
    print ply_fields

    if len(ply_shp) > 1:
        # using python list comprehension syntax
        # shapely asShape to convert to shapely geom
        ply_list = [asShape(feature) for feature in ply_shp]

        # create new shapely multipolygon
        out_multi_ply = MultiPolygon(ply_list)

        # # equivalent to the 2 lines above without using list comprehension
        # new_feature_list = []
        # for feature in features:
        #     temp = asShape(feature)
        #     new_feature_list.append(temp)
        # out_multi_ply = MultiPolygon(new_feature_list)

        print "converting to MultiPolygon: " + str(out_multi_ply)
    else:
        print "one or no features found"
        shply_ply = asShape(ply_shp)
        out_multi_ply = MultiPolygon(shply_ply)

    return out_multi_ply


def create_union(in_ply1, in_ply2, result_geojson):
    """
    Create union polygon
    :param in_ply1: first input shapely polygon
    :param in_ply2: second input shapely polygon
    :param result_geojson: output geojson file including full file path
    :return: shapely MultiPolygon
    """
    # union the polygon outer linestrings together
    outer_bndry = in_ply1.boundary.union(in_ply2.boundary)

    # rebuild linestrings into polygons
    output_poly_list = polygonize(outer_bndry)

    out_geojson = dict(type='FeatureCollection', features=[])

    # generate geojson file output
    for (index_num, ply) in enumerate(output_poly_list):
        feature = dict(type='Feature', properties=dict(id=index_num))
        feature['geometry'] = ply.__geo_interface__
        out_geojson['features'].append(feature)

    # create geojson file on disk
    json.dump(out_geojson, open(result_geojson, 'w'))

    # create shapely MultiPolygon
    ply_list = []
    for fp in polygonize(outer_bndry):
        ply_list.append(fp)

    out_multi_ply = MultiPolygon(ply_list)

    return out_multi_ply


def write_wkt(filepath, features):
    """

    :param filepath: output path for new JavaScript file
    :param features: shapely geometry features
    :return:
    """
    with open(filepath, "w") as f:
        # create a JavaScript variable called ply_data used in html
        # Shapely dumps geometry out to WKT
        f.write("var ply_data = '" + dumps(features) + "'")


def output_geojson_fc(shply_features, outpath):
    """
    Create valid GeoJSON python dictionary
    :param shply_features: shapely geometries
    :param outpath:
    :return: GeoJSON FeatureCollection File
    """

    new_geojson = []
    for feature in shply_features:
        feature_geom_geojson = feature.__geo_interface__
        myfeat = Feature(geometry=feature_geom_geojson,
                         properties={'name': "mojo"})
        new_geojson.append(myfeat)

    out_feat_collect = FeatureCollection(new_geojson)

    with open(outpath, "w") as f:
        f.write(json.dumps(out_feat_collect))


if __name__ == "__main__":

    # define our inputs
    shp1 = realpath("../geodata/temp1-ply.shp")
    shp2 = realpath("../geodata/temp2-ply.shp")

    # define outputs
    out_geojson_file = realpath("../geodata/res_union.geojson")
    output_union = realpath("../geodata/output_union.geojson")
    out_wkt_js = realpath("ol3/data/results_union.js")

    # create our shapely multipolygons for geoprocessing
    in_ply_1_shape = create_shapes(shp1)
    in_ply_2_shape = create_shapes(shp2)

    # run generate union function
    result_union = create_union(in_ply_1_shape, in_ply_2_shape, out_geojson_file)

    # write to our output js file the new polygon as wkt
    write_wkt(out_wkt_js, result_union)

    # write the results out to well known text (wkt) with shapely dump
    geojson_fc = output_geojson_fc(result_union, output_union)

Results can then be opened in the /ch06/code/ol3/html/ch06-02_union.html folder with a double-click to start them in your local web browser. You should see the following web map if everything's gone smoothly:

<img src="./50790OS_06_05.jpg" height=400 width=400>

### How it works...

A quick high-level run-through of what is going on here at the beginning should help clear the air. We have four functions and nine variables within our Python code to split the load of input and output data. The running of our code takes place in the if __name__ == "main": call that is found at the end of the code. We start defining two variables to deal with our inputs that we are going to union together. These two are our input Shapefiles and the other three outputs are GeoJSON and JavaScript files.

The create_shapes()function converts our Shapefile into Shapely MultiPolygon geometry objects. Inside the Python class, the list comprehension is used to generate a new list of polygon objects, which are the input list of polygons used to create our output MultiPolygon. Next, we'll simply run this function passing in our input Shapefiles.

Our create_union() function is up next where we do the real union work. We begin by unioning the two geometry boundaries together that produces a union set of LineStrings and represents the outer bounds of our input polygons. The reason for this is that we do not want to lose the geometries of both polygons, which will, by default, dissolve into one big polygon when simply passed into the Shapely union function. Therefore, we need to rebuild the polygons with the polygonize() Shapely function.

The polygonize function creates a Python generator object, not a simple geometry. This is an iterator that's similar to a list that we need to loop over to get at the individual polygons it's created for us.

We do exactly this in the next code segment using the enumerate() Python function that automatically creates an ID for us for each feature that we use as the id field in the attribute results. After our loop, we use the standard Python json.dump() method to export our newly created GeoJSON file and write it to disk using the Python open() method in the write mode.

Lastly, in our create_union() function, we prepare to output our resulting union polygon as a Shapely MultiPolygon object. This is accomplished simply by looping through the polygonize() iterator and outputting a list that feeds into the Shapely MultiPolygon()function. Finally, we execute the union function, passing in our two input geometries and specifying the output GeoJSON file.

So, we can view our results in our web map as we did in the previous exercise using a small function called write_wkt(). This little function takes the file path to the output JavaScript file that we want to create and the MultiPolygon result's geometry. Shapely then dumps the geometry into the Well Known Text format as we write it out to the JavaScript file.

In the end, a small function called output_geojson_fc() is used to output another GeoJSON file, this time using the Python geojson library. This simply shows you another way to recreate a GeoJSON file. Since GeoJSON is a plain text file, it is possible to create it in many unique ways depending on your personal programming preference.

## 6.3. Union polygons with merging (dissolving)

To demonstrate what merging is all about, we will take an example out of the NOAA weather data. It provides an awesome minute-by-minute update of Shapefiles to satisfy your desire to download data. We will look at a week's collection of weather warnings and union these warnings together, giving us the total warning area issued in this week.

A conceptual visualization of our desired results is shown here:

<img src="./50790OS_06_06.jpg" height=400 width=400>

Most of the data is located around Florida, but has some polygons near Hawaii and California. To see the original data or find new data, check out these links:

<ul>
    <li>http://www.nws.noaa.gov/geodata/catalog/wsom/html/pubzone.htm</li>
    <li>http://nws.noaa.gov/regsci/gis/week.html</li>
    <li>http://www.nws.noaa.gov/geodata/index.html</li>
</ul>

If you want to see the state boundaries, you can find them at https://www.census.gov/geo/maps-data/data/cbf/cbf_state.html.

Here is what a sample of the data looks like around Florida before the union, which is visualized with QGIS:

<img src="./50790OS_06_07.jpg" height=400 width=400>

In [ ]:
# #!/usr/bin/env python
# -*- coding: utf-8 -*-
from shapely.geometry import MultiPolygon
from shapely.ops import cascaded_union
from os.path import realpath
from utils import create_shapes
from utils import out_geoj
from utils import write_wkt


def check_geom(in_geom):
    """
    :param in_geom: input valid Shapely geometry objects
    :return: Shapely MultiPolygon cleaned
    """
    plys = []
    for g in in_geom:
        # if geometry is NOT valid
        if not g.is_valid:
            print "Oh no invalid geometry"
            # clean polygon with buffer 0 distance trick
            new_ply = g.buffer(0)
            print "now lets make it valid"
            # add new geometry to list
            plys.append(new_ply)
        else:
            # add valid geometry to list
            plys.append(g)
    # convert new polygons into a new MultiPolygon
    out_new_valid_multi = MultiPolygon(plys)
    return out_new_valid_multi


if __name__ == "__main__":

    # input NOAA Shapefile
    shp = realpath("../geodata/temp-all-warn-week.shp")

    # output union_dissolve results as GeoJSON
    out_geojson_file = realpath("../geodata/ch06-03_union_dissolve.geojson")

    out_wkt_js = realpath("ol3/data/ch06-03_results_union.js")

    # input Shapefile and convert to Shapely geometries
    shply_geom = create_shapes(shp)

    # Check the Shapely geometries if they are valid if not fix them
    new_valid_geom = check_geom(shply_geom)

    # run our union with dissolve
    dissolve_result = cascaded_union(new_valid_geom)

    # output the resulting union dissolved polygons to GeoJSON file
    out_geoj(dissolve_result, out_geojson_file)

    write_wkt(out_wkt_js, dissolve_result)

Your resulting web map will look like this:

<img src="./50790OS_06_08.jpg" height=400 width=400>

### How it works...

We are starting to increasingly reuse more code that is now tucked away in our /ch06/code/utils.py module. As you see in the imports, we use three functions for the standard input and output of data. The main application starts with defining our NOAA input Shapefile and defining the output GeoJSON file. Then, if we run the code, it will crash due to data validity issues. So, we create a new function to check our input data for invalid geometries. This new function will catch these invalid geometries and convert them to valid polygons.

Shapely has a geometry property called is_valid, which accesses the GEOS engine to check for geometry validity based on the simple features in the OGC specification.

The reason for these anomalies is that when data is overlaid and processed, geometries become combined or cut at angles that are not always optimal.

At last, we have clean data to work with, making the rest of our journey very simple by running the Shapely cascaded_union() function, which will dissolve all our overlapping polygons. Our resulting MultiPolygons are pushed further into our out_geoj() function, which finally writes the new geometries to disk in our /ch06/geodata folder.

## 6.4. Performing an identity function (difference + intersection)

In ESRI geoprocessing terminology, there is an overlay function called identity. This is a very useful function to call when you want to keep all the original geometry boundaries of ONLY the input features combined with an intersection of input features.

<img src="./50790OS_06_09.jpg" height=400 width=400>

This boils down to a formula that calls for both difference and intersect. We first find the difference (input feature - intersection), then add the intersection to create our results as follows:

<code>(input feature – intersection) + intersection = result</code>

### How to do it...

In [ ]:
##!/usr/bin/env python
# -*- coding: utf-8 -*-
from shapely.geometry import asShape, MultiPolygon
from utils import shp2_geojson_obj, out_geoj, write_wkt
from os.path import realpath

def create_polys(shp_data):
    """
    :param shp_data: input GeoJSON
    :return: MultiPolygon Shapely geometry
    """
    plys = []
    for feature in shp_data['features']:
        shape = asShape(feature['geometry'])
        plys.append(shape)

    new_multi = MultiPolygon(plys)
    return new_multi


def create_out(res1, res2):
    """

    :param res1: input feature
    :param res2: identity feature
    :return: MultiPolygon identity results
    """
    identity_geoms = []

    for g1 in res1:
        identity_geoms.append(g1)
    for g2 in res2:
        identity_geoms.append(g2)

    out_identity = MultiPolygon(identity_geoms)
    return out_identity


if __name__ == "__main__":
    # out two input test Shapefiles
    shp1 = realpath("../geodata/temp1-ply.shp")
    shp2 = realpath("../geodata/temp2-ply.shp")

    # output resulting GeoJSON file
    out_geojson_file = realpath("../geodata/result_identity.geojson")

    output_wkt_identity = realpath("ol3/data/ch06-04_results_identity.js")


    # convert our Shapefiles to GeoJSON
    # then to python dictionaries
    shp1_data = shp2_geojson_obj(shp1)
    shp2_data = shp2_geojson_obj(shp2)

    # transform our GeoJSON data into Shapely geom objects
    shp1_polys = create_polys(shp1_data)
    shp2_polys = create_polys(shp2_data)

    # run the difference and intersection
    res_difference = shp1_polys.difference(shp2_polys)
    res_intersection = shp1_polys.intersection(shp2_polys)

    # combine the difference and intersection polygons into results
    result_identity = create_out(res_difference, res_intersection)

    # export identity results to a GeoJSON
    out_geoj(result_identity, out_geojson_file)

    # write out new JavaScript variable with wkt geometry
    write_wkt(output_wkt_identity, result_identity )

The resulting polygons can now be visualized in your browser. Now simply open the /ch06/code/ol3/html/ch06-04_identity.html file and you will see this map:

<img src="./50790OS_06_10.jpg" height=400 width=400>

### How it works...

We have hidden away a couple of gems in our util.py utilities file called shp2_geojson_obj and out_geoj. The first one takes in our Shapefile and returns a Python dictionary object. Our function actually creates a valid GeoJSON in the form of a Python dictionary that could very easily be converted to a JSON string using the standard json.dumps()Python module.

With this overhead out of the way, we can jump into creating Shapely geometries that can be used for our analysis. The create_polys()function does exactly this: it takes in our geometries, returning a MultiPolygon. This MultiPolygon is used to calculate our difference and intersection.

So, at last, we can do the analysis calculation starting with the Shapely difference function using our temp1-ply.shp as our input feature and temp2-poly.shp as our identity feature. The difference function only returns the geometries of the input features that do not intersect the other feature. Next up, we execute the intersection function that only returns geometries that overlap between our two inputs.

Our recipe is almost completed; we only need to combine these two new results to produce our new identity result's MultiPolygon. The create_out()function takes two arguments, the first being our input features and the second is our resulting intersection features. The order is very important; otherwise your results will be reversed. So make sure that you enter the correct order of input.

We run through each of the geometries and combine them into a fancy new MultiPolygon called result_identity. This is then pumped into our out_geoj() function, which writes out a new GeoJSON file to your /ch06/geodata/ folder.

Our out_geoj() function is located in the utils.py file and might need a quick explanation. The input is a list of geometries and the file path of the output GeoJSON file location on disk. We simply create a new dictionary, and then loop through each geometry, exporting the Shapely geometry to a GeoJSON file using the built-in Shapely __geo_interface__.

<pre><strong>Note</strong>

If you want to read up on the __geo_interface__, do so for yourself and find out what it is and why it's so cool at https://gist.github.com/sgillies/2217756.
</pre>

For those of you looking for the two utility functions, here they are for your reading pleasure:

In [1]:
def shp2_geojson_obj(shapefile_path):
    # open shapefile
    in_ply = shapefile.Reader(shapefile_path)
    # get a list of geometry and records
    shp_records = in_ply.shapeRecords()
    # get list of fields excluding first list object
    fc_fields = in_ply.fields[1:]

    # using list comprehension to create list of field names
    field_names = [field_name[0] for field_name in fc_fields ]
    my_fc_list = []
    # run through each shape geometry and attribute
    for x in shp_records:
        field_attributes = dict(zip(field_names, x.record))
        geom_j = x.shape.__geo_interface__
        my_fc_list.append(dict(type='Feature', geometry=geom_j,
                               properties=field_attributes))

    geoj_json_obj = {'type': 'FeatureCollection',
                    'features': my_fc_list}

    return geoj_json_obj
def out_geoj(list_geom, out_geoj_file):
    out_geojson = dict(type='FeatureCollection', features=[])

    # generate geojson file output
    for (index_num, ply) in enumerate(list_geom):
        feature = dict(type='Feature', properties=dict(id=index_num))
        feature['geometry'] = ply.__geo_interface__
        out_geojson['features'].append(feature)

    # create geojson file on disk
    json.dump(out_geojson, open(out_geoj_file, 'w'))

